In [1]:
#%run -i read_counts.py
all_freqs = pickle.load(open('all_freqs.pkl', 'rb'))

In [3]:
from matplotlib import pyplot as plt

%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')

import pandas as pd
pd.options.display.mpl_style = 'default'

from mpltools import style
from mpltools import layout

style.use('ggplot')

## see: https://stackoverflow.com/questions/19536817/manipulate-html-module-font-size-in-ipython-notebook
class sizeme():
    """ Class to change html fontsize of object's representation"""
    def __init__(self,ob, size, height=100):
        self.ob = ob
        self.size = size
        self.height = height
    def _repr_html_(self):
        repl_tuple = (self.size, self.height, self.ob._repr_html_())
        return u'<span style="font-size:{0}%; line-height:{1}%">{2}</span>'.format(*repl_tuple)

In [4]:
## see https://stackoverflow.com/questions/14656852/how-to-use-pandas-dataframes-and-numpy-arrays-in-rpy2
## and http://ipython.org/ipython-doc/rel-0.13/config/extensions/rmagic.html
## note there's a ri2pandas() to convert back.
## but note, rpy2 2.4.0 and later automagically translates dataframes: 
## https://stackoverflow.com/questions/20630121/pandas-how-to-convert-r-dataframe-back-to-pandas
%load_ext rpy2.ipython
#import rpy2.robjects.pandas2ri as p2r
#rdf = p2r.pandas2ri(info)
#%Rpush rdf

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [5]:
x = all_freqs['Desulfovibrio_vulgaris_Hildenborough_uid57645'].copy()
sample_info = pd.read_excel('Sample_Info_COMPLETE.xlsx') ##,skiprows=[0])
sample_info = sample_info.set_index( sample_info['Sample name'] )
sample_infos = { k:sample_info.ix[all_freqs[k].columns.droplevel(1).values] for k in all_freqs.keys() }
info = sample_infos['Desulfovibrio_vulgaris_Hildenborough_uid57645'].copy()
sizeme(info.head(3),50,120)

,Sample name,Source,Barcode,Strain/condition,cultivation type,Chemostat/batch ID #,Description/condition details,Description/condition details -2,carbon source,electron donor,concentration (mM),electron acceptor,growth rate per h,T0C,organisms
Sample name,,,,,,,,,,,,,,,
Sample_US-1505885,Sample_US-1505885,Labcorp,US-1505885,PS 37 C log 2,batch,NaN,"pyruvate 80mM, sulfate 15mM 37C",sulfate respiration/growth,pyruvate 80mM,pyruvate,NaN,sulfate,NaN,37,D vulgaris Hildenborugh
Sample_US-1505888,Sample_US-1505888,Labcorp,US-1505888,LS 37C log 1,batch,NaN,"lactate 40mM, sulfate15mM 37C",sulfate respiration/growth,lactate 40mM,lactate,NaN,sulfate,NaN,37,D vulgaris Hildenborugh
Sample_US-1505895,Sample_US-1505895,Labcorp,US-1505895,LS 37C e. stat 1,batch,NaN,"lactate 40mM, sulfate15mM 37C",fermentation,lactate 10mM,lactate,10,none,0,37,D vulgaris Hildenborugh


In [9]:
%Rpush info
%Rpush x
genes = x.index.values.astype('str')
print(genes)
%Rpush genes
#%R save.image("Untitled1.RData")

['DVU0001' 'DVU0002' 'DVU0003' ..., '__too_low_aQual' '__not_aligned'
 '__alignment_not_unique']


In [10]:
%%R
#    load("Untitled1.RData")
    gr=info$growth.rate.per.h; gr=as.numeric(as.character(gr)); gr[is.na(gr)]=0.2; info$growth.rate.per.h=gr
    model.mat = model.matrix(~-1+cultivation.type+electron.donor+electron.acceptor+growth.rate.per.h+T0C,data=info)
    good_gene = which(!substr(genes,1,2)=='__' & apply(x,1,sum)>0)
    x = x[good_gene,]
    rownames(x)=genes[good_gene]
    x=log10(t(as.matrix(x)))
    x[is.na(x)|is.infinite(x)] <- 0
    require(sparseLDA)
    X <- normalize(x); X <- X$Xc
    Y <- model.mat[ ,grepl('donor',colnames(model.mat))]
    Y <- cbind(Y, electron.donorhydrogen=1-apply(Y,1,sum))
    sda.fit=sda(X,Y,stop=-9,trace=T)
    betas <- sda.fit$beta; rownames(betas) <- sda.fit$varNames; betas <- as.data.frame(betas)
    varNames <- sda.fit$varNames

ite:  1  ridge cost:  28.35699  |b|_1:  5.342971 
ite:  2  ridge cost:  25.22139  |b|_1:  4.855164 
ite:  3  ridge cost:  22.42048  |b|_1:  3.913895 
ite:  4  ridge cost:  13.16292  |b|_1:  5.125011 
ite:  5  ridge cost:  25.14276  |b|_1:  8.103719 
ite:  6  ridge cost:  33.62026  |b|_1:  8.984382 
ite:  7  ridge cost:  38.82663  |b|_1:  8.589824 
ite:  8  ridge cost:  39.20047  |b|_1:  8.875339 
ite:  9  ridge cost:  39.31899  |b|_1:  8.956233 
ite:  10  ridge cost:  39.3529  |b|_1:  8.978692 
ite:  11  ridge cost:  39.37749  |b|_1:  8.97961 
ite:  12  ridge cost:  39.38487  |b|_1:  8.979826 
ite:  13  ridge cost:  39.38704  |b|_1:  8.97989 
ite:  14  ridge cost:  39.38768  |b|_1:  8.979909 
ite:  15  ridge cost:  39.38787  |b|_1:  8.979914 
ite:  16  ridge cost:  39.38793  |b|_1:  8.979916 
ite:  17  ridge cost:  39.38794  |b|_1:  8.979916 
ite:  1  ridge cost:  30.14872  |b|_1:  5.119144 
ite:  2  ridge cost:  30.14872  |b|_1:  5.119144 
final update, total ridge cost:  69.53667  |b

In [11]:
%Rpull betas varNames
print(varNames)

['DVU0031' 'DVU0177' 'DVU0650' 'DVU0745' 'DVU1013' 'DVU1520' 'DVU1917'
 'DVU2204' 'DVU2293' 'DVU2451' 'DVU2650' 'DVU2652' 'DVU2666' 'DVU2667'
 'DVU3027' 'DVU3029' 'DVU3031' 'Dv16SC']


submit these to network portal to see function [link](http://networks.systemsbiology.net/search?q=DVU0024+DVU0031+DVU0032+DVU0037+DVU0177+DVU0181+DVU0434+DVU0650+DVU0745+DVU0747+DVU1013+DVU1098+DVU1142+DVU1520+DVU1553+DVU1637+DVU1859+DVU1917+DVU2204+DVU2249+DVU2250+DVU2293+DVU2451+DVU2594+DVU2622+DVU2652+DVU2666+DVU2667+DVU2742+DVU2952+DVU2981+DVU2982+DVU3027+DVU3029+DVU3031+DVU3291+DVU3292+DVU3339.1+%22DVU_tRNA-Ala-4%22+Dv16SC)



In [12]:
from IPython.display import IFrame
IFrame('http://networks.systemsbiology.net/search?q=DVU0024+DVU0031+DVU0032+DVU0037+DVU0177+DVU0181+DVU0434+DVU0650+DVU0745+DVU0747+DVU1013+DVU1098+DVU1142+DVU1520+DVU1553+DVU1637+DVU1859+DVU1917+DVU2204+DVU2249+DVU2250+DVU2293+DVU2451+DVU2594+DVU2622+DVU2652+DVU2666+DVU2667+DVU2742+DVU2952+DVU2981+DVU2982+DVU3027+DVU3029+DVU3031+DVU3291+DVU3292+DVU3339.1+%22DVU_tRNA-Ala-4%22+Dv16SC', 
       width='100%', height=150)

In [13]:
%%R
    gr <- info$growth.rate.per.h
    Y <- cbind(no_growth=as.integer(gr==0), fast_growth=as.integer(gr>=0.2), med_growth=as.integer(gr>0&gr<0.2))
    sda.fit=sda(X,Y,stop=-9,trace=T)
    betas <- sda.fit$beta; rownames(betas) <- sda.fit$varNames; betas <- as.data.frame(betas)
    varNames <- sda.fit$varNames

ite:  1  ridge cost:  38.50368  |b|_1:  3.669826 
ite:  2  ridge cost:  43.41143  |b|_1:  5.084331 
ite:  3  ridge cost:  49.36249  |b|_1:  5.862193 
ite:  4  ridge cost:  58.38915  |b|_1:  4.653358 
ite:  5  ridge cost:  56.73306  |b|_1:  5.017014 
ite:  6  ridge cost:  55.74344  |b|_1:  5.210932 
ite:  7  ridge cost:  55.12528  |b|_1:  5.324631 
ite:  8  ridge cost:  59.20131  |b|_1:  4.586446 
ite:  9  ridge cost:  59.18413  |b|_1:  4.591013 
ite:  10  ridge cost:  59.16968  |b|_1:  4.594459 
ite:  11  ridge cost:  55.11451  |b|_1:  5.326566 
ite:  12  ridge cost:  59.20198  |b|_1:  4.586256 
ite:  13  ridge cost:  59.18471  |b|_1:  4.590869 
ite:  14  ridge cost:  59.17016  |b|_1:  4.594351 
ite:  15  ridge cost:  55.1116  |b|_1:  5.327089 
ite:  16  ridge cost:  59.20216  |b|_1:  4.586204 
ite:  17  ridge cost:  59.18486  |b|_1:  4.59083 
ite:  18  ridge cost:  59.17029  |b|_1:  4.594322 
ite:  19  ridge cost:  55.11081  |b|_1:  5.327231 
ite:  20  ridge cost:  59.20221  |b|_1:  4

In [14]:
%Rpull betas varNames
#print(varNames)
query = '%22'+'%22+%22'.join(varNames)+'%22'
from IPython.display import IFrame
IFrame('http://networks.systemsbiology.net/search?q='+query, 
       width='100%', height=150)